# Interactive Assessment Application

## Getting Started with Streamlit in Snowflake (SiS)

### *Pre-requisites:*
    
- An application to run this notebook file like [VSCode](https://code.visualstudio.com/) or [Jupyter](https://jupyter.org/)

- Specify your Snowflake credentials updating the following file ```./iaa_config.toml``` required by snowflakecli.

**Notice**: The Snowflake Cli will be used for this deployments process.

### Python3.11 Requirement 

In [ ]:
! chmod +x ./pre_req_install_python_3.sh
! bash ./pre_req_install_python_3.sh

### Snowflake CLI Requirement

In [ ]:
! python3 ./pre_req_install_snow_3.py

In [ ]:
! snow --info
! snow --version

### Snowflake Account Configuration

In [ ]:
from IPython.display import display, HTML

warning_message = "<div style='padding: 20x; background-color: yellow; color: black; border: 1px solid red;'>Warning! Please Check your config .toml file before to continue</div>"
display(HTML(warning_message))

```toml
    [connections.name]
    account = "<Snowflake Account Identifier>"
    user = "<Snowflake Username>"
    password = "<Snowflake password, if not using key/pair>"
    database = "<Snowflake database>"
    schema = ""<Snowflake schema>""
    warehouse = ""<Snowflake warehouse>""
    role = ""<Snowflake role>""
```

#### *Note*

For MacOS and Linux systems, Snowflake cli requires the ```iaa_config.toml``` file to limit its file permissions to read and write for the file owner only. To set the file required file permissions execute the following commands: [more info](https://docs.snowflake.com/en/developer-guide/snowflake-cli/connecting/specify-credentials#add-snowflake-credentials-using-a-configuration-file)


#### Snowflake Toml local file

In [ ]:
! if [[ $(uname -s) == "Linux" || $(uname -s) == "Darwin" ]]; then chmod 0600 "iaa_config.toml" ; fi
! pwd && ls -la iaa_config.toml

#### Validate Connection Entries

Run the following command for getting information about the new app.

In [ ]:
! chmod +x ./pre_req_validate_iaa_config_toml.py
! python ./pre_req_validate_iaa_config_toml.py

#### Check Point Connection to Snowflake

Run the following command for checking a valid Snowflake connection

In [ ]:
! snow --config-file ./iaa_config.toml sql -q "Select CURRENT_ORGANIZATION_NAME();Select CURRENT_ACCOUNT_NAME();" -c iaa

## Deployment to SiS

### Configure local environment

### Get Connection Entries

Run the following command for getting information about the new app configured before.

In [ ]:
import os
import toml
import yaml

config = toml.load("iaa_config.toml")
user_value = config["connections"]["iaa"]["user"]
os.environ["MY_USER"] = user_value
database_value = config["connections"]["iaa"]["database"]
os.environ["MY_DATABASE"] = database_value
schema_value = config["connections"]["iaa"]["schema"]
os.environ["MY_SCHEMA"] = schema_value
warehouse_value = config["connections"]["iaa"]["warehouse"]
os.environ["MY_WAREHOUSE"] = warehouse_value

file_path = os.path.join("../streamlit/", "snowflake.yml")
with open(file_path, "r") as file:
    config = yaml.safe_load(file)

app_main_file = config["entities"]["streamlit_app"]["main_file"]
os.environ["MY_APP_MAIN"] = app_main_file
app_name = config["entities"]["streamlit_app"]["identifier"]["name"]
os.environ["MY_APP"] = app_name
app_title = config["entities"]["streamlit_app"]["title"]
os.environ["MY_APP_TITLE"] = app_title
app_stage = config["entities"]["streamlit_app"]["stage"]
os.environ["MY_APP_STAGE"] = app_stage
app_wh = config["entities"]["streamlit_app"]["query_warehouse"]
os.environ["MY_APP_WH"] = app_wh

print(f"MAIN FILE:  {os.environ['MY_APP_MAIN']}")
print(f"APP NAME:   {os.environ['MY_APP']}")
print(f"APP TITLE:  {os.environ['MY_APP_TITLE']}")

### Deploying Streamlit to SiS by snowflakecli

Configure the Snowflake user for deployment process

In [ ]:
print(f"MY USER NAME        : {os.environ['MY_USER']}")

!snow --config-file ../deployment/iaa_config.toml sql -q "alter user {os.environ['MY_USER']} set MINS_TO_BYPASS_MFA=20;" -c iaa
!snow --config-file ../deployment/iaa_config.toml sql -q "describe user {os.environ['MY_USER']};" -c iaa

#### Deployment the App Schema to SiS

In [ ]:
! chmod +x ./pre_req_schema_config.sh

MY_DEPLOY_DB = !(grep 'database' iaa_config.toml | cut -d'=' -f2 | tr -d ' "')
MY_DEPLOY_SCHEMA = !(grep 'schema' iaa_config.toml | cut -d'=' -f2 | tr -d ' "')
MY_DEPLOY_STAGE = !(grep 'stage' ../streamlit/snowflake.yml | awk '{print $2}')
MY_DEPLOY_WAREHOUSE = !(grep 'warehouse' iaa_config.toml | cut -d'=' -f2 | tr -d ' "')

! bash ./pre_req_schema_config.sh "{MY_DEPLOY_DB[0]}" "{MY_DEPLOY_SCHEMA[0]}" "{MY_DEPLOY_STAGE[0]}" "{MY_DEPLOY_WAREHOUSE[0]}"
! snow --config-file ../deployment/iaa_config.toml sql -q "CREATE DATABASE IF NOT EXISTS {MY_DEPLOY_DB[0]};" -c iaa
! snow --config-file ../deployment/iaa_config.toml sql --filename ../schema/APP_SCHEMA.sql -c iaa;

#### Deployment the Maps Stage to SiS

In [ ]:
! chmod +x ./upload_mappings.py
! python ./upload_mappings.py

#### Deployment the APP to SiS

In [ ]:
! pwd && ls -la ../deployment/iaa_config.toml
! cd ../streamlit && snow --config-file ../deployment/iaa_config.toml streamlit deploy --replace --verbose -c iaa

#### Refresh Deployment

In [ ]:
! snow --config-file ./iaa_config.toml sql -q "ALTER TASK UPDATE_IAA_TABLES_TASK RESUME;ALTER TASK REFRESH_MAPPINGS_CORE_TASK RESUME;CALL REFRESH_MANUAL_MAPPINGS();" -c iaa

## Getting Application Information

In [ ]:
import subprocess
import re
import os

result = subprocess.check_output(
    f"snow --config-file ../deployment/iaa_config.toml streamlit get-url {os.environ['MY_DATABASE']}.{os.environ['MY_SCHEMA']}.{os.environ['MY_APP']} -c iaa",
    shell=True,
).decode("UTF-8")
ansi_escape = re.compile(r"(?:\x1B[@-_]|[\x80-\x9F])[0-?]*[ -/]*[@-~]")
clean_url = ansi_escape.sub("", result)
os.environ["IAA_APP_URL"] = clean_url
print(clean_url)

In [ ]:
from IPython.display import display as displayHTML, HTML

URL = os.environ["IAA_APP_URL"]
html = f'<html><div  style="display:flex;justify-content: center;"><a href="{URL}"><button style="background-color:#249edc;color: #fff;border:1px solid #249edc;cursor:pointer;border-radius:45px;font-weight:800;line-height:18px;padding: 8px 16px" type="button">Open IAA</button></a></div></html>'
displayHTML(HTML(html))